In [2]:
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import plotly.graph_objects as go
import datetime
import pandas as pd
import csv
import os
import time
from dateutil import tz
import glob
import quandl as q
import re
import plotly.graph_objects as go
import numpy as np
import streamlit as st
from plotly.subplots import make_subplots

st.set_page_config(layout="wide")
# %%
from fredapi import Fred

In [3]:
api_key_fred = os.environ['api_key_fred']
fred = Fred(api_key=api_key_fred)

quandl_api_key = os.environ['quandl_api_key']


In [4]:
# # # start - read in BTC data # # #
datasource_btcusd = "BITFINEX/DOGEUSD.csv"
# dogeusd_data = pd.read_csv("coindata/{}".format(
#     datasource_btcusd.replace("/", " ")), index_col=0)
# dogeusd_data.index = pd.to_datetime(dogeusd_data.index)

# most_recent_stored_btcusd_date = dogeusd_data.sort_index().tail(1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")

data = q.get(datasource_btcusd.split(".")[0],   start_date='2017-01-01',
                 end_date='{}'.format(todays_date),
                 api_key=quandl_api_key)
data.info()
data["First"] = data.Last.shift(1)
data = data.dropna()
data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 820 entries, 2021-04-21 to 2023-07-19
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   High    820 non-null    float64
 1   Low     820 non-null    float64
 2   Mid     820 non-null    float64
 3   Last    820 non-null    float64
 4   Bid     820 non-null    float64
 5   Ask     820 non-null    float64
 6   Volume  820 non-null    float64
dtypes: float64(7)
memory usage: 51.2 KB


,High,Low,Mid,Last,Bid,Ask,Volume,First
Date,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120
2021-04-24,0.290000,0.228900,0.270640,0.271600,0.270350,0.270930,3.672039e+07,0.243570
2021-04-25,0.289400,0.225320,0.250750,0.250880,0.250350,0.251150,1.529839e+07,0.271600
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880
...,...,...,...,...,...,...,...,...
2023-07-15,0.075200,0.067782,0.071906,0.071784,0.071881,0.071931,1.243599e+07,0.068273
2023-07-16,0.072264,0.069055,0.069996,0.069970,0.069981,0.070011,5.000303e+06,0.071784
2023-07-17,0.073321,0.067711,0.069638,0.069659,0.069615,0.069660,6.300208e+06,0.069970


In [5]:
dogeusd_data = data.sort_index()
# store current df with up-to-date values
dogeusd_data.to_csv('coindata/{}'.format(
    datasource_btcusd.replace("/", " ")), index=True)

In [6]:
# # # start - data processing # # #
dogeusd_data = dogeusd_data.dropna()
dogeusd_data["350_movingaverage"] = pd.Series.rolling(
    dogeusd_data["Last"], window=350, min_periods=1).mean()
dogeusd_data["111_movingaverage"] = pd.Series.rolling(
    dogeusd_data["Last"], window=111, min_periods=1).mean()
# # # end - data processing # # #


In [7]:
# run_it = st.sidebar.button('Show visualizations')

# TODO: Future integrate ETH
# display_name_all_twitter_user_scraped_csvs, all_twitter_user_scraped_csvs = get_all_stored_crypto_csvs()
# display_name_user_selection_list_containing_twitter_user = st.sidebar.selectbox(
#     "Select existing Twitter-User", list(display_name_all_twitter_user_scraped_csvs), 0)


In [8]:
# # # start - get data from fred api and store csv # # #
# # GET S&P DATA
# Billions of Dollars 
data_SP500 = fred.get_series('SP500')
data_SP500 = data_SP500.sort_index()
# store current df with up-to-date values
data_SP500.to_csv('coindata/data_SP500.csv', index=True)

# # Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW) Millions of Dollars
data_WALCL = fred.get_series('WALCL')
data_WALCL.dropna()
data_WALCL = data_WALCL.sort_index()
# store current df with up-to-date values
data_WALCL.to_csv('coindata/data_WALCL.csv', index=True)

# # Overnight Reverse Repurchase Agreements Treasury Securities Sold by the Federal Reserve in the Temporary Open Market Operations (RRPONTSYD)
data_RRPONTSYD = fred.get_series('RRPONTSYD')
#Assets: Total Assets: Total Assets: Wednesday Level (RESPPANWW)
# Billions of U.S. Dollars
# data_FRED_RRPONTSYD.values = data_FRED_RRPONTSYD.values * 1000 
data_RRPONTSYD = data_RRPONTSYD * 1000 
data_RRPONTSYD.dropna()
data_RRPONTSYD = data_RRPONTSYD.sort_index()
# store current df with up-to-date values
data_RRPONTSYD.to_csv('coindata/data_RRPONTSYD.csv', index=True)

# # Deposits with Federal Reserve Banks, other than Reserve Balances: U.S. Treasury, General Account (WTREGEN)
# Billions of Dollars 
data_FRED_WTREGEN = fred.get_series('WTREGEN')
data_FRED_WTREGEN = data_FRED_WTREGEN * 1000

data_FRED_WTREGEN.dropna()
data_FRED_WTREGEN = data_FRED_WTREGEN.sort_index()
# store current df with up-to-date values
data_FRED_WTREGEN.to_csv('coindata/data_FRED_WTREGEN.csv', index=True)

# # net liquidity vs s&p500 weekly - Diagram
datasource_fred_total_assets = "data_WALCL.csv"
fred_total_assets = pd.read_csv("coindata/{}".format(
    datasource_fred_total_assets.replace("/", " ")), index_col=0)
fred_total_assets.index = pd.to_datetime(fred_total_assets.index)

most_recent_stored_fred_rrpontsyd_date = fred_total_assets.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")
todays_date = datetime.date.today() - timedelta(days=1)
todays_date = todays_date.strftime("%Y-%m-%d")

datasource_FRED_WTREGEN = "data_FRED_WTREGEN.csv"
FRED_WTREGEN_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_WTREGEN.replace("/", " ")), index_col=0)
FRED_WTREGEN_data.index = pd.to_datetime(FRED_WTREGEN_data.index)

most_recent_stored_FRED_WTREGEN_date = FRED_WTREGEN_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

datasource_FRED_RRPONTSYD = "data_RRPONTSYD.csv"
FRED_RRPONTSYD_data = pd.read_csv("coindata/{}".format(
    datasource_FRED_RRPONTSYD.replace("/", " ")), index_col=0)
FRED_RRPONTSYD_data.index = pd.to_datetime(FRED_RRPONTSYD_data.index)

most_recent_stored_FRED_RRPONTSYD_date = FRED_RRPONTSYD_data.sort_index().tail(
    1).index[0].strftime("%Y-%m-%d")

# # # end - get data from fred api and store csv # # #

In [9]:
dogeusd_data = dogeusd_data[(dogeusd_data.index > '2020-08-11 00:00:00')]

In [10]:
import warnings
warnings.filterwarnings("ignore")
from astral import moon
import datetime

# take the data_SP500 index and iterate over it to get the moon phase for each day
# the make a new df out of it called df_moon_phase
df_moon_phase = pd.DataFrame()
for i in data_SP500.index:
    df_moon_phase = df_moon_phase.append({"date": i, "moon_phase": moon.phase(i)}, ignore_index=True)

df_moon_phase["moon_phase"] = df_moon_phase["moon_phase"].astype(int)
df_moon_phase["date"] = pd.to_datetime(df_moon_phase["date"])
df_moon_phase["date"] = df_moon_phase["date"].dt.date
df_moon_phase = df_moon_phase.set_index("date")

# check the relationship between moon phase and dogeusd_data
# merge sp500 with moon phase
dogeusd_data_and_moon_phase = pd.merge(
    dogeusd_data, df_moon_phase, left_index=True, right_index=True, how='left')
dogeusd_data_and_moon_phase = dogeusd_data_and_moon_phase.dropna()
dogeusd_data_and_moon_phase["350_movingaverage"] = pd.Series.rolling(
    dogeusd_data_and_moon_phase["Last"], window=350, min_periods=1).mean()
dogeusd_data_and_moon_phase["111_movingaverage"] = pd.Series.rolling(
    dogeusd_data_and_moon_phase["Last"], window=111, min_periods=1).mean()

# calculat the daily price change for dogeusd_data_and_moon_phase in percentage
dogeusd_data_and_moon_phase["daily_price_change"] = dogeusd_data_and_moon_phase["Last"].pct_change()
dogeusd_data_and_moon_phase["daily_price_change"] = dogeusd_data_and_moon_phase["daily_price_change"].fillna(0)
# calculat the daily price change for dogeusd_data_and_moon_phase in absolut values
dogeusd_data_and_moon_phase["daily_price_change_abs"] = dogeusd_data_and_moon_phase["Last"].diff()
dogeusd_data_and_moon_phase["daily_price_change_abs"] = dogeusd_data_and_moon_phase["daily_price_change_abs"].fillna(0)
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs
Date,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-13,0.071749,0.064686,0.070068,0.070009,0.070055,0.070081,8.336210e+06,0.064666,0.082703,0.075161,24.0,0.082625,0.005343
2023-07-14,0.073231,0.066240,0.068178,0.068273,0.068160,0.068196,1.256866e+07,0.070009,0.082567,0.075039,25.0,-0.024797,-0.001736
2023-07-17,0.073321,0.067711,0.069638,0.069659,0.069615,0.069660,6.300208e+06,0.069970,0.082440,0.074938,27.0,0.020301,0.001386


In [11]:
# now we want the relationship of the moon_phase column on the daily_price_change and daily_price_change_abs column
# we group by the moon_phase column and calculate the mean of the daily_price_change and daily_price_change_abs column
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("moon_phase").mean()
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
dogeusd_data_and_moon_phase_grouped


,moon_phase,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,daily_price_change,daily_price_change_abs
0,0.0,0.167800,0.150045,0.157203,0.157192,0.157128,0.157278,2.155073e+07,0.162338,0.227809,0.186009,-0.003364,-0.003345
1,1.0,0.140844,0.124019,0.135327,0.135267,0.135226,0.135428,2.117920e+07,0.133925,0.199749,0.156990,0.002893,0.001202
2,2.0,0.152936,0.137474,0.146827,0.146818,0.146756,0.146897,2.253241e+07,0.143427,0.212504,0.172835,0.002437,0.001919
3,3.0,0.148682,0.136326,0.141745,0.141782,0.141682,0.141808,2.535434e+07,0.140278,0.217220,0.172345,0.013685,0.001907
4,4.0,0.153300,0.141289,0.146592,0.146630,0.146525,0.146658,1.701858e+07,0.149049,0.216571,0.171548,-0.012722,-0.004704
5,5.0,0.154730,0.142796,0.146880,0.146884,0.146818,0.146941,1.484241e+07,0.150324,0.214530,0.174594,-0.018292,-0.004141
6,6.0,0.160513,0.138183,0.148387,0.148298,0.148293,0.148481,2.928444e+07,0.155047,0.228851,0.185747,0.015764,-0.004162
7,7.0,0.157296,0.140192,0.149764,0.149777,0.149694,0.149833,1.838475e+07,0.149264,0.220321,0.176747,0.001757,0.002490
8,8.0,0.138916,0.124096,0.129949,0.129945,0.129885,0.130012,1.940491e+07,0.133731,0.193070,0.150150,-0.006534,-0.003000
9,9.0,0.147430,0.131682,0.137733,0.137843,0.137651,0.137814,2.051483e+07,0.142692,0.207666,0.168664,-0.016312,-0.001835


In [12]:
# # plot the moon_phase vs the daily_price_change as seperate lines with the time on the x-axis
# fig_moon_phase = make_subplots(specs=[[{"secondary_y": True}]])
# fig_moon_phase.add_trace(
#     go.Scatter(
#         x=dogeusd_data_and_moon_phase_grouped["moon_phase"],
#         y=dogeusd_data_and_moon_phase_grouped["daily_price_change"],
#         name="daily_price_change",
#         mode='lines',
#         marker=dict(color="red"),
#     )
# )

# fig_moon_phase

In [13]:
dogeusd_data_and_moon_phase["validate_ta_logic_category"] = ""
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] > 11) | (dogeusd_data_and_moon_phase["moon_phase"] <= 3), "validate_ta_logic_category"] = "long"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >3) & (dogeusd_data_and_moon_phase["moon_phase"] <=11), "validate_ta_logic_category"] = "short"
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category
Date,,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000,short
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550,short
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540,long
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100,long
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-13,0.071749,0.064686,0.070068,0.070009,0.070055,0.070081,8.336210e+06,0.064666,0.082703,0.075161,24.0,0.082625,0.005343,long
2023-07-14,0.073231,0.066240,0.068178,0.068273,0.068160,0.068196,1.256866e+07,0.070009,0.082567,0.075039,25.0,-0.024797,-0.001736,long
2023-07-17,0.073321,0.067711,0.069638,0.069659,0.069615,0.069660,6.300208e+06,0.069970,0.082440,0.074938,27.0,0.020301,0.001386,long


In [14]:
# 0 .. 6.99	New moon
# 7 .. 13.99	First quarter
# 14 .. 20.99	Full moon
# 21 .. 27.99	Last quarter
# create a new column called moon_phase_category and fill it with the moon_phase_category
dogeusd_data_and_moon_phase["moon_phase_category"] = ""
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 0) & (dogeusd_data_and_moon_phase["moon_phase"] <= 6.99), "moon_phase_category"] = "New moon"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 7) & (dogeusd_data_and_moon_phase["moon_phase"] <= 13.99), "moon_phase_category"] = "First quarter"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 14) & (dogeusd_data_and_moon_phase["moon_phase"] <= 20.99), "moon_phase_category"] = "Full moon"
dogeusd_data_and_moon_phase.loc[(dogeusd_data_and_moon_phase["moon_phase"] >= 21) & (dogeusd_data_and_moon_phase["moon_phase"] <= 27.99), "moon_phase_category"] = "Last quarter"
dogeusd_data_and_moon_phase

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category,moon_phase_category
Date,,,,,,,,,,,,,,,
2021-04-22,0.310190,0.254200,0.263865,0.266120,0.263390,0.264340,3.914806e+07,0.308760,0.266120,0.266120,9.0,0.000000,0.000000,short,First quarter
2021-04-23,0.268080,0.179490,0.243185,0.243570,0.242950,0.243420,7.169683e+07,0.266120,0.254845,0.254845,10.0,-0.084736,-0.022550,short,First quarter
2021-04-26,0.280660,0.248000,0.269230,0.269110,0.268930,0.269530,1.969569e+07,0.250880,0.259600,0.259600,13.0,0.104857,0.025540,long,First quarter
2021-04-27,0.280080,0.265080,0.271455,0.271210,0.271230,0.271680,1.280031e+07,0.269110,0.262503,0.262503,14.0,0.007804,0.002100,long,Full moon
2021-04-28,0.344710,0.255940,0.318290,0.318830,0.318070,0.318510,4.295062e+07,0.271210,0.273768,0.273768,15.0,0.175583,0.047620,long,Full moon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-13,0.071749,0.064686,0.070068,0.070009,0.070055,0.070081,8.336210e+06,0.064666,0.082703,0.075161,24.0,0.082625,0.005343,long,Last quarter
2023-07-14,0.073231,0.066240,0.068178,0.068273,0.068160,0.068196,1.256866e+07,0.070009,0.082567,0.075039,25.0,-0.024797,-0.001736,long,Last quarter
2023-07-17,0.073321,0.067711,0.069638,0.069659,0.069615,0.069660,6.300208e+06,0.069970,0.082440,0.074938,27.0,0.020301,0.001386,long,Last quarter


In [15]:
# group the dogeusd_data_and_moon_phase by the moon_phase_category column and calculate the mean of the daily_price_change and daily_price_change_abs column
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("moon_phase_category").mean()
dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
dogeusd_data_and_moon_phase_grouped

,moon_phase_category,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs
0,First quarter,0.149326,0.132578,0.140598,0.140617,0.140530,0.140666,2.594062e+07,0.141417,0.213162,0.171595,9.972789,-0.002799,-0.000565
1,Full moon,0.146837,0.133102,0.140058,0.140058,0.139992,0.140124,1.971324e+07,0.139732,0.210325,0.168805,17.020408,0.001491,0.000793
2,Last quarter,0.161608,0.143686,0.153359,0.153378,0.153277,0.153442,1.832457e+07,0.152551,0.216252,0.175070,23.904762,0.001503,0.000043
3,New moon,0.154372,0.138800,0.146368,0.146356,0.146292,0.146444,2.171374e+07,0.148025,0.216977,0.174487,2.916667,0.000080,-0.001637


In [16]:
# # group the dogeusd_data_and_moon_phase by the moon_phase_category column and calculate the mean of the daily_price_change and daily_price_change_abs column
# dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase.groupby("validate_ta_logic_category").mean()
# dogeusd_data_and_moon_phase_grouped = dogeusd_data_and_moon_phase_grouped.reset_index()
# dogeusd_data_and_moon_phase_grouped
# validate_ta_logic_category	High	Low	Mid	Last	Bid	Ask	Volume	First	350_movingaverage	111_movingaverage	moon_phase	daily_price_change	daily_price_change_abs
# 0	long	0.154273	0.138621	0.146826	0.146832	0.146752	0.146901	1.979144e+07	0.146288	0.214739	0.173295	15.864286	0.001697	0.000425
# 1	short	0.149864	0.132988	0.140668	0.140678	0.140597	0.140739	2.557094e+07	0.143205	0.212704	0.170402	7.509091	-0.004078	-0.002268

In [17]:
# Correlation Analysis: First, it would be good to calculate the correlation coefficient between the moon_phase_category and the daily_price_change
# we use the pandas function corr() to calculate the correlation coefficient
dogeusd_data_and_moon_phase["daily_price_change"].corr(dogeusd_data_and_moon_phase["moon_phase"])

0.028057304037777373

In [18]:
# Regression Analysis: A linear regression model could be used to predict the BTC price change based on the moon phase. If the model is statistically significant (which you can check using a hypothesis test for the model parameters), this would suggest that there is a relationship. Moreover, the regression coefficients can be interpreted as the expected change in the BTC price for each phase of the moon.
# we use the statsmodels package to run the regression
import statsmodels.api as sm
# we need to add a constant term to the data set to estimate the intercept of the linear regression model
X = sm.add_constant(dogeusd_data_and_moon_phase["moon_phase"])
# we run the regression
model = sm.OLS(dogeusd_data_and_moon_phase["daily_price_change"], X)
results = model.fit()
# we print the summary of the regression
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:     daily_price_change   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.4593
Date:                Thu, 20 Jul 2023   Prob (F-statistic):              0.498
Time:                        23:40:15   Log-Likelihood:                 741.03
No. Observations:                 585   AIC:                            -1478.
Df Residuals:                     583   BIC:                            -1469.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0031      0.006     -0.569      0.5

In [19]:
#                          OLS Regression Results                              
# ==================================================================================
# Dep. Variable:     daily_price_change_abs   R-squared:                       0.007
# Model:                                OLS   Adj. R-squared:                  0.006
# Method:                     Least Squares   F-statistic:                     5.527
# Date:                    Thu, 20 Jul 2023   Prob (F-statistic):             0.0190
# Time:                            16:08:07   Log-Likelihood:                -6671.4
# No. Observations:                     765   AIC:                         1.335e+04
# Df Residuals:                         763   BIC:                         1.336e+04
# Df Model:                               1                                         
# Covariance Type:                nonrobust                                         
# ==============================================================================
#                  coef    std err          t      P>|t|      [0.025      0.975]
# ------------------------------------------------------------------------------
# const       -186.6173    104.328     -1.789      0.074    -391.421      18.186
# moon_phase    15.6265      6.647      2.351      0.019       2.578      28.675
# ==============================================================================
# Omnibus:                       77.255   Durbin-Watson:                   1.994
# Prob(Omnibus):                  0.000   Jarque-Bera (JB):              478.758
# Skew:                          -0.151   Prob(JB):                    1.09e-104
# Kurtosis:                       6.864   Cond. No.                         30.6
# ==============================================================================

# Notes:
# [1] Standard Errors assume that the covariance matrix of the errors is correctly specified.

# Your regression output indicates that the moon phase variable is significant at the 5% level (since the p-value, denoted as "P>|t|" for "moon_phase",
# is less than 0.05). This means there is evidence to suggest that the moon phase has some influence on the absolute daily price change of Bitcoin.

# However, be careful in interpreting the magnitude and meaning of this relationship. The coefficient for "moon_phase" is 15.6265, 
# which means that for each unit increase in moon phase, the model predicts an average increase of 15.6265 units in the absolute daily price change of Bitcoin. 
# But because moon phase is probably a categorical variable coded as numbers, it might not make much sense to think about "unit increases" in this way.

# Moreover, the R-squared value of the model is quite low (0.007), meaning that the moon phase only explains a very small portion of the variation 
# in the absolute daily price change of Bitcoin. This suggests that there are many other factors influencing Bitcoin's daily price change, 
# and that the moon phase alone is not a strong predictor.

# Also, it's important to note that this is a simple linear regression analysis, and it doesn't account for potential confounding variables or 
# other complexities of financial time-series data, such as autocorrelation or non-stationarity.

# It's also worth mentioning again that correlation does not imply causation. Just because there is a statistically significant 
# relationship between moon phase and Bitcoin's price change, it doesn't mean the moon phase is causing these price changes.
# It could be due to chance or other variables that happen to correlate with the moon phase.

In [20]:
dogeusd_data_and_moon_phase.head()

,High,Low,Mid,Last,Bid,Ask,Volume,First,350_movingaverage,111_movingaverage,moon_phase,daily_price_change,daily_price_change_abs,validate_ta_logic_category,moon_phase_category
Date,,,,,,,,,,,,,,,
2021-04-22,0.31019,0.25420,0.263865,0.26612,0.26339,0.26434,3.914806e+07,0.30876,0.266120,0.266120,9.0,0.000000,0.00000,short,First quarter
2021-04-23,0.26808,0.17949,0.243185,0.24357,0.24295,0.24342,7.169683e+07,0.26612,0.254845,0.254845,10.0,-0.084736,-0.02255,short,First quarter
2021-04-26,0.28066,0.24800,0.269230,0.26911,0.26893,0.26953,1.969569e+07,0.25088,0.259600,0.259600,13.0,0.104857,0.02554,long,First quarter
2021-04-27,0.28008,0.26508,0.271455,0.27121,0.27123,0.27168,1.280031e+07,0.26911,0.262503,0.262503,14.0,0.007804,0.00210,long,Full moon
2021-04-28,0.34471,0.25594,0.318290,0.31883,0.31807,0.31851,4.295062e+07,0.27121,0.273768,0.273768,15.0,0.175583,0.04762,long,Full moon


In [30]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Normalizing the 'Volume'
dogeusd_data_and_moon_phase['norm_volume'] = (dogeusd_data_and_moon_phase['Volume'] - dogeusd_data_and_moon_phase['Volume'].min()) / (dogeusd_data_and_moon_phase['Volume'].max() - dogeusd_data_and_moon_phase['Volume'].min())

# Calculate daily price change
dogeusd_data_and_moon_phase['daily_change'] = dogeusd_data_and_moon_phase['Last'].diff()

# Assign colors based on positive or negative change
dogeusd_data_and_moon_phase['color'] = np.where(dogeusd_data_and_moon_phase['daily_change'] > 0, 'green', 'red')

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase['Last'],
        mode='markers+lines',
        marker=dict(
            size=dogeusd_data_and_moon_phase['norm_volume']*30,  # Scale marker size
            color=dogeusd_data_and_moon_phase['color']  # Assign color based on 'daily_change'
        ),
        name='Last'
    ),
    secondary_y=False
)

fig.add_trace(
    go.Bar(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase['Volume'],
        name='Volume',
        marker=dict(color='blue')
    ),
    secondary_y=True
)

# Add figure title and labels
fig.update_layout(
    title_text="DOGEUSD Last and Volume",
    xaxis_title="Date",
    yaxis_title="Last",
    yaxis2_title="Volume",
    xaxis_rangeslider_visible=True,
)

# Show the plot
fig.show()


In [22]:
asd

NameError: name 'asd' is not defined

In [ ]:
# make a new plot and plot the daily_price_change of btc by time on the x-axis
fig_btc_price_change = make_subplots(specs=[[{"secondary_y": True}]])
fig_btc_price_change.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase["Last"],
        name="Last",
        mode='lines',
        marker=dict(color="red"),
    )
)

# now add the moon_phase column as a second y-axis
fig_btc_price_change.add_trace(
    go.Scatter(
        x=dogeusd_data_and_moon_phase.index,
        y=dogeusd_data_and_moon_phase["moon_phase"],
        name="moon_phase",
        mode='lines',
        marker=dict(color="blue"),
    ),
    secondary_y=True,
)